<a href="https://colab.research.google.com/github/peiwen880923/text-mining/blob/main/LDA%20model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
pip install pyLDAvis==3.2.1

In [3]:
import numpy as np
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

#spacy
import spacy
import nltk
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

/usr/local/lib/python3.7/dist-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()
/usr/local/lib/python3.7/dist-packages/jsonschema/compat.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import MutableMapping, Sequence  # noqa
/usr/local/lib/python3.7/dist-packages/jsonschema/compat.py:6: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import MutableMapping, Sequence  # noqa
/usr/local/lib/python3.7/dist-packages/nltk/decorators.py:70: DeprecationWarning: `formatargspec` is deprecated since Python 3.5. Use `signature` and the `Signature` object directly
  formatva

In [4]:
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f) 
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

In [5]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [6]:
stopwords = stopwords.words("english")

In [7]:
print (stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [8]:
data = load_data("ushmm_dn.json")["texts"]

print (data[0][0:90])

 My name David Kochalski. I was born in a small town called , and I was born May 5, 1928. 


In [9]:
from nltk.corpus import stopwords

In [10]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    for text in texts:
        doc = nlp(text)
        new_text = []
        for token in doc:
            if token.pos_ in allowed_postags:
                new_text.append(token.lemma_)
        final = " ".join(new_text)
        texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data)
print (lemmatized_texts[0][0:90])

name bear small town call bear very hard work child father mother small mill flour buckwhe


In [11]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0][0:20])

['name', 'bear', 'small', 'town', 'call', 'bear', 'very', 'hard', 'work', 'child', 'father', 'mother', 'small', 'mill', 'flour', 'buckwheat', 'prosperous', 'comfortable', 'go', 'school']


In [12]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)

[(0, 2), (1, 10), (2, 1), (3, 2), (4, 1), (5, 2), (6, 1), (7, 2), (8, 3), (9, 1), (10, 12), (11, 1), (12, 8), (13, 1), (14, 2), (15, 1), (16, 3), (17, 2), (18, 1), (19, 2)]
able


In [13]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=30,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha="auto")

In [14]:
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:248: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3      0.299007  0.056224       1        1  39.901675
26     0.408926  0.063311       2        1  27.767836
19     0.363363  0.093261       3        1  10.229889
21     0.448390  0.105332       4        1   9.313174
5      0.327235 -0.040072       5        1   9.020515
0      0.435806 -0.136746       6        1   2.032227
10     0.272609  0.265085       7        1   0.868075
6      0.333209 -0.215615       8        1   0.685883
17     0.016176 -0.275327       9        1   0.080221
11    -0.207996 -0.261528      10        1   0.054161
8     -0.171584  0.167274      11        1   0.036564
4     -0.137798  0.000679      12        1   0.004218
18    -0.133985  0.009606      13        1   0.000896
1     -0.132539  0.010167      14        1   0.000398
23    -0.132538  0.009897      15        1   0.000285
25    -0.132538  0.009897      16        1   0.000284
13    -0.132538  0.009897      17        1   0.000283
28    -0.132546  0.009897      18        1   0.000274
16    -0.132553  0.009897      19        1   0.000269
14    -0.132552  0.009897      20        1   0.000269
27    -0.132552  0.009897      21        1   0.000267
15    -0.132553  0.009897      22        1   0.000267
29    -0.132553  0.009897      23        1   0.000265
7     -0.132554  0.009897      24        1   0.000264
2     -0.132555  0.009897      25        1   0.000264
12    -0.132555  0.009897      26        1   0.000263
9     -0.132556  0.009897      27        1   0.000258
20    -0.132558  0.009897      28        1   0.000253
24    -0.132558  0.009897      29        1   0.000253
22    -0.132560  0.009897      30        1   0.000251, topic_info=       Term          Freq         Total Category  logprob  loglift
751      so  20550.000000  20550.000000  Default  30.0000  30.0000
867    very  11216.000000  11216.000000  Default  29.0000  29.0000
350      go  24069.000000  24069.000000  Default  28.0000  28.0000
601  people  12578.000000  12578.000000  Default  27.0000  27.0000
170    come  14045.000000  14045.000000  Default  26.0000  26.0000
..      ...           ...           ...      ...      ...      ...
822   think      0.000161   5446.384518  Topic30  -9.6310  -4.4375
819    then      0.000161   7974.200900  Topic30  -9.6310  -4.8188
67     back      0.000161   5057.134442  Topic30  -9.6310  -4.3634
486    look      0.000161   3446.839993  Topic30  -9.6311  -3.9801
133     can      0.000161   5030.896958  Topic30  -9.6311  -4.3583

[1973 rows x 6 columns], token_table=       Topic      Freq   Term
term                         
14186     11  1.023899   abet
0          1  0.580656   able
0          2  0.100447   able
0          3  0.200895   able
0          4  0.002643   able
...      ...       ...    ...
919        4  0.097171  young
919        5  0.008260  young
919        6  0.017977  young
919        7  0.009717  young
919        8  0.033524  young

[2154 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 27, 20, 22, 6, 1, 11, 7, 18, 12, 9, 5, 19, 2, 24, 26, 14, 29, 17, 15, 28, 16, 30, 8, 3, 13, 10, 21, 25, 23])

In [15]:
import pandas as pd
print(pd.__version__)


1.3.1


In [16]:
!pip install pandas==1.3.1

In [17]:
!pip install pandas --upgrade


  Using cached pandas-1.3.4-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (11.3 MB)
  Attempting uninstall: pandas
    Found existing installation: pandas 1.3.1
    Uninstalling pandas-1.3.1:
      Successfully uninstalled pandas-1.3.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas~=1.1.0; python_version >= "3.0", but you have pandas 1.3.4 which is incompatible.
